# P3Alpha

### Import libraries

In [44]:
import pandas as pd
import numpy as np
from scipy import sparse
import os
import csv

from utilities import load_sparse_matrix

### Load dataset

In [45]:
ICM = load_sparse_matrix("icm.npz")
URM = load_sparse_matrix("urm.npz")
URM_test = load_sparse_matrix("urm_test.npz")
URM_train = load_sparse_matrix("urm_train.npz")
URM_val = load_sparse_matrix("urm_val.npz")
URM_train_val = load_sparse_matrix("urm_train_val.npz")

Loaded urm.npz
Loaded urm_test.npz
Loaded urm_train.npz
Loaded urm_val.npz
Loaded urm_train_val.npz
Loaded icm.npz


In [46]:
from src.Evaluation.Evaluator import EvaluatorHoldout

evaluator_val = EvaluatorHoldout(URM_val, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 744 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 320 ( 0.8%) Users that have less than 1 test interactions


In [47]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(5, 1000),
    "alpha": Real(0, 1),
}

In [48]:
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = P3alphaRecommender

hyperparameterSearch = SearchBayesianSkopt(
    recommender_class,
    evaluator_validation=evaluator_val,
    evaluator_test=evaluator_test)

In [49]:
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_val],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

In [50]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 10  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [51]:
hyperparameterSearch.search(
    recommender_input_args,
    recommender_input_args_last_test = recommender_input_args_last_test,
    hyperparameter_search_space = hyperparameters_range_dictionary,
    n_cases = n_cases,
    n_random_starts = n_random_starts,
    save_model = "last",
    output_folder_path = output_folder_path, # Where to save the results
    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
    metric_to_optimize = metric_to_optimize,
    cutoff_to_optimize = cutoff_to_optimize,
)

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 379, 'alpha': 0.17823009600176645}
P3alphaRecommender: URM Detected 3461 (12.4%) items with no interactions.
P3alphaRecommender: Similarity column 27968 (100.0%), 778.08 column/sec. Elapsed time 35.94 sec
EvaluatorHoldout: Processed 40885 (100.0%) in 1.46 min. Users per second: 466
SearchBayesianSkopt: New best config found. Config 0: {'topK': 379, 'alpha': 0.17823009600176645} - results: PRECISION: 0.0342057, PRECISION_RECALL_MIN_DEN: 0.0573706, RECALL: 0.0536303, MAP: 0.0147594, MAP_MIN_DEN: 0.0247339, MRR: 0.1138607, NDCG: 0.0532587, F1: 0.0417702, HIT_RATE: 0.2615385, ARHR_ALL_HITS: 0.1292034, NOVELTY: 0.0032531, AVERAGE_POPULARITY: 0.3995350, DIVERSITY_MEAN_INTER_LIST: 0.6344807, DIVERSITY_HERFINDAHL: 0.9634465, COVERAGE_ITEM: 0.1620423, COVERAGE_ITEM_HIT: 0.0145523, ITEMS_IN_GT: 0.8711742, COVERAGE_USER: 0.9821278, COVERAGE_USER_HIT: 0.2568642, USERS_IN_GT: 0.9821278, DIVER